# Implementing Apriori Algorithm

Import Libraries

In [1]:
##Importing Libraries
import re
import warnings
import pandas as pd
import json
import numpy as np
#import pyfpgrowth
from efficient_apriori import apriori
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import udf
import pyspark.sql.functions as func
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.fpm import FPGrowth
import datetime
import time

Create Spark Session

In [ ]:
sc = SparkContext.getOrCreate()
spark1 = SparkSession(sc)

Load Dataset

In [ ]:
df = spark1.read.csv("D:\Data Mining and Machine Learning 2\Project\BigTempRb.txt", sep=',',header='true', inferSchema='true',escape="\"" )
#df.dtypes

Select columns and check row count

In [ ]:
df = df.select("RequestTimestamp","ResponseRgBasketId","RequestSiteId",
      "RequestBasketValue","RequestBasketId",
      "RequestNumberBasketItems","RequestBasketJsonString")

df.count()

Create ItemList column

In [ ]:
### UDF to extract item list from json
def get_item_list(item_json):
    #print(item_json)
    response = json.loads(item_json)
    lister = []
    for nest in response['items']:
        lister.append(nest["b"])
    return (list(set(lister)))

### converting above function to a UDF
item_list = udf(get_item_list, ArrayType(StringType()))

## Creating item_list from JsonString
df = df.withColumn('ItemList', item_list(df.RequestBasketJsonString))
#df.show()

#load ItemList to transactions list
transactions = df.select("ItemList").rdd.flatMap(lambda x: x).collect()

Build Apriori

In [ ]:
## input paramters
input_support = .0002 ## VALUE RANGES BETWEEN 0 AND 1
input_confidence = 0.001 ## VALUE RANGES BETWEEN 0 AND 1
############### Applying Pyspark FPgrowth algorithm ######################
#fpGrowth = FPGrowth(itemsCol="ItemList", minSupport=input_support, minConfidence=input_confidence)
#model = fpGrowth.fit(df)
itemsets, rules = apriori(transactions, min_support=input_support, min_confidence=input_confidence)


Print rules for validation purpose

In [ ]:
# Print out every rule with 1 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule)  # Prints the rule and its confidence, support, lift, ...
    


# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift): 
  print(rule)  # Prints the rule and its confidence, support, lift, ...